**Ringkasan dan tujuan**

Sel ini memberikan ringkasan singkat alur pipeline dan tujuan penelitian (fine‑tune ResNet untuk regresi aflatoksin). Baca sebelum menjalankan notebook.

# Finetune ResNet — Pipeline ringkas (tf.keras)

Ringkasan singkat:
- Tujuan: fine-tune backbone ResNet + head untuk regresi aflatoksin.
- Alur yang disarankan:
  1. Kumpulkan dan bagi data secara stratified (train/val/test).
  2. (Opsional) Oversample hanya pada TRAIN dengan resampling filepath.
  3. Gunakan tf.data dengan augment on-the-fly untuk TRAIN.
  4. Phase 1: latih head dengan backbone dibekukan → simpan bobot backbone (weights-only).
  5. Phase 2 (opsional): jalankan Keras Tuner; setiap trial memuat bobot backbone dari Phase 1 ke instansi backbone sebelum menempelkan head.
  6. Latih akhir dengan HP terbaik → evaluasi & simpan model akhir.

Catatan cepat:
- Pastikan `base_weights_path` dihasilkan oleh fungsi `save_base_weights()` pada Phase 1.
- Jalankan sel-sel notebook sesuai urutan: dari atas ke bawah.
- Edit hanya sel parameter di bagian akhir sebelum menjalankan seluruh pipeline.

Cara pakai singkat:
- Buka sel parameter (akhir), atur `DATA_DIR`, `OUTPUT_DIR`, `DO_TUNING`, lalu jalankan setiap sel secara berurutan.


### Outline fungsi (struktur per-fungsi)

Berikut daftar fungsi utama yang tersedia di notebook ini beserta tujuan singkat, input, dan output yang diharapkan:

- `collect_image_paths(data_dir: Path, pattern='*.png')`
  - Tujuan: kumpulkan semua path gambar dan label (nama folder) dari `data_dir`.
  - Input: `data_dir` (Path) — direktori dataset berstruktur folder per-kelas.
  - Output: `(filepaths: np.ndarray, labels: np.ndarray)`

- `stratified_split(filepaths, labels, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)`
  - Tujuan: bagi dataset secara stratified sesuai label.
  - Input: arrays filepaths & labels.
  - Output: tuples `(fp_train, lb_train), (fp_val, lb_val), (fp_test, lb_test)`

- `oversample_filepaths(fp_train, lb_train, target_count=None, seed=42)`
  - Tujuan: oversample (resample filepaths) hanya untuk TRAIN agar distribusi kelas seimbang.
  - Input: train filepaths & labels, target_count per kelas.
  - Output: `(new_files, new_labels)` arrays

- `labels_to_ppb(labels)`
  - Tujuan: konversi label string ke nilai float (ppb) bila perlu.
  - Output: `np.array` float32 shape `(N,)` atau `(N,1)` saat digunakan downstream.

- `build_dataset_from_paths(filepaths, labels_ppb, img_size=(224,224), batch_size=32, augment=False, shuffle=False)`
  - Tujuan: membuat `tf.data.Dataset` siap dipakai untuk training/val/test.
  - Input: filepaths array dan labels float.
  - Output: `tf.data.Dataset` yielding `(image, label)` batched.

- `build_regression_model(input_shape=(224,224,3), backbone='resnet50', units_l1=512, dropout_rate=0.3, base_trainable=False)`
  - Tujuan: buat model tf.keras regresi dan kembalikan `(model, base_model)`.
  - Output: `model` (komplet) dan `base_model` (backbone) untuk disimpan/di-load terpisah.

- `train_phase1(model, train_ds, val_ds, initial_epochs=30, lr=1e-6, output_dir=Path('.'))`
  - Tujuan: latih head dengan backbone dibekukan, simpan best Phase 1.
  - Output: `history` training

- `save_base_weights(base_model, path)`
  - Tujuan: simpan bobot backbone (weights-only) untuk digunakan tuner/final.
  - Output: file `.h5` di `path`.

- `run_keras_tuner(train_ds, val_ds, base_weights_path, max_trials=20, executions_per_trial=1, output_dir=Path('.'))`
  - Tujuan: jalankan Keras Tuner. Untuk setiap trial, fungsi membuat backbone baru dan memuat `base_weights_path` sebelum menempelkan head.
  - Output: `(tuner, best_hp)`

- `final_train_with_best_hp(base_weights_path, best_hp, train_ds, val_ds, initial_epochs=30, fine_tune_epochs=40, output_dir=Path('.'))`
  - Tujuan: bangun model final memakai `best_hp`, muat bobot backbone, atur `fine_tune_at`, latih, dan simpan model terbaik.
  - Output: `(model, history)`

- `evaluate_model(model, test_ds)`
  - Tujuan: evaluasi model pada test set dan kembalikan metrics `{mae,mse,rmse,r2}`.

Catatan: gunakan sel Parameter di akhir notebook untuk mengontrol eksekusi per tahapan (RUN_* flags).

**Imports & quick environment check**

Sel ini mengimpor library yang diperlukan. Pastikan `tensorflow`, `keras-tuner` (opsional), `scikit-learn`, dan `pandas` terpasang di environment Anda.

In [1]:
# 1) Imports and quick environment check
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Optional: Keras Tuner (install if needed)
try:
    import keras_tuner as kt
except Exception:
    kt = None

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('TensorFlow:', tf.__version__, 'Keras Tuner:', kt is not None)


TensorFlow: 2.16.1 Keras Tuner: True


**Utilities**

Fungsi utilitas: mengumpulkan path gambar, stratified split (train/val/test), oversample (train saja), dan konversi label ke nilai numerik (ppb).

In [2]:
# 2) Utilities: collect, stratified split, oversample (train only), label mapping

def collect_image_paths(data_dir: Path, pattern='*.png'):
    data_dir = Path(data_dir)
    filepaths = []
    labels = []
    for class_folder in sorted([p for p in data_dir.iterdir() if p.is_dir()]):
        for img in class_folder.glob(pattern):
            filepaths.append(str(img))
            labels.append(class_folder.name)
    return np.array(filepaths), np.array(labels)


def stratified_split(filepaths, labels, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6
    fp_train, fp_temp, lb_train, lb_temp = train_test_split(filepaths, labels, train_size=train_ratio, stratify=labels, random_state=seed)
    val_size = val_ratio / (val_ratio + test_ratio)
    fp_val, fp_test, lb_val, lb_test = train_test_split(fp_temp, lb_temp, train_size=val_size, stratify=lb_temp, random_state=seed)
    return (fp_train, lb_train), (fp_val, lb_val), (fp_test, lb_test)


def oversample_filepaths(fp_train, lb_train, target_count=None, seed=42):
    rng = np.random.default_rng(seed)
    classes, counts = np.unique(lb_train, return_counts=True)
    class_to_files = {c: fp_train[lb_train == c].tolist() for c in classes}
    if target_count is None:
        target_count = int(max(counts))
    new_files = []
    new_labels = []
    for c in classes:
        files = class_to_files[c]
        n = len(files)
        if n >= target_count:
            chosen = rng.choice(files, size=target_count, replace=False)
        else:
            chosen = rng.choice(files, size=target_count, replace=True)
        new_files.extend(chosen.tolist())
        new_labels.extend([c] * target_count)
    combined = list(zip(new_files, new_labels))
    rng.shuffle(combined)
    new_files, new_labels = zip(*combined)
    return np.array(new_files), np.array(new_labels)


def labels_to_ppb(labels):
    try:
        return np.array([float(l) for l in labels], dtype=np.float32)
    except Exception:
        unique = sorted(list(set(labels)))
        mapping = {c: float(i + 1) for i, c in enumerate(unique)}
        return np.array([mapping[l] for l in labels], dtype=np.float32)


**tf.data builder**

Membangun pipeline `tf.data.Dataset` untuk membaca file path, decoding gambar, `preprocess_input`, augmentasi (opsional untuk train), batching, dan prefetching.

In [3]:
# 3) tf.data builder: parse, augment (train only), batch, prefetch

def build_dataset_from_paths(filepaths, labels_ppb, img_size=(224,224), batch_size=32, augment=False, shuffle=False):
    AUTOTUNE = tf.data.AUTOTUNE
    paths = tf.constant(filepaths.tolist()) if isinstance(filepaths, np.ndarray) else tf.constant(list(filepaths))
    labels = tf.constant(labels_ppb.tolist()) if isinstance(labels_ppb, np.ndarray) else tf.constant(list(labels_ppb))
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(filepaths), seed=None)

    def _parse_func(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.resize(image, img_size)
        image = tf.cast(image, tf.float32)
        image = preprocess_input(image)
        label = tf.cast(label, tf.float32)
        label = tf.expand_dims(label, axis=-1)
        return image, label

    ds = ds.map(_parse_func, num_parallel_calls=AUTOTUNE)

    if augment:
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomRotation(0.05),
            layers.RandomZoom(0.05),
        ], name='data_augmentation')

        def _augment(image, label):
            return data_augmentation(image), label

        ds = ds.map(_augment, num_parallel_calls=AUTOTUNE)

    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds


**Penjelasan sebelum sel Model builder**

Sel ini mendefinisikan arsitektur model regresi: membuat backbone (ResNet50), menambahkan GlobalAveragePooling, lapisan Dense + Dropout, dan output tunggal. Fungsi mengembalikan tuple `(model, base_model)` sehingga `base_model` (backbone) dapat disimpan atau dimuat secara terpisah.

In [4]:
# 4) Model builder: return (model, base_model) so base_model can be saved/loaded separately

def build_regression_model(input_shape=(224,224,3), backbone='resnet50', units_l1=512, dropout_rate=0.3, base_trainable=False):
    if backbone.lower() == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError('Unsupported backbone. Use resnet50 for now.')
    base_model.trainable = base_trainable
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units_l1, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear', name='aflatoxin_output')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model, base_model


**Penjelasan sebelum sel Phase 1 (train head)**

Sel ini berisi fungsi `train_phase1()` untuk melatih head dengan backbone dibekukan. Callback penting: EarlyStopping, ReduceLROnPlateau, dan ModelCheckpoint (menyimpan model terbaik Phase 1). Juga ada `save_base_weights()` untuk menyimpan bobot backbone sebagai file `.h5`.

In [5]:
# 5) Phase 1: train head (backbone frozen) and save backbone weights

def train_phase1(model, train_ds, val_ds, initial_epochs=30, lr=1e-6, output_dir=Path('.')):
    callbacks = [
        EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-8, verbose=1),
        ModelCheckpoint(str(output_dir / 'best_phase1.keras'), monitor='val_loss', save_best_only=True, verbose=1)
    ]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae','mse'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=initial_epochs, callbacks=callbacks, verbose=1)
    return history


def save_base_weights(base_model, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    base_model.save_weights(str(path))


**Penjelasan sebelum sel Keras Tuner**

Sel ini menyiapkan integrasi Keras Tuner. Fungsi `run_keras_tuner()` membuat backbone baru untuk tiap trial, memuat `base_weights_path` yang disimpan dari Phase 1, kemudian menambah head yang hyperparameternya dicari. Pastikan `DO_TUNING` hanya diaktifkan bila `keras-tuner` terpasang.

In [6]:
# 6) Keras Tuner integration (each trial loads backbone weights into backbone)

def run_keras_tuner(train_ds, val_ds, base_weights_path, max_trials=20, executions_per_trial=1, output_dir=Path('.')):
    if kt is None:
        raise RuntimeError('keras-tuner is not installed. Install keras-tuner to run tuning.')

    def build_model_for_tuning(hp):
        # build backbone and load weights saved from Phase1 (weights-only file)
        base_model = ResNet50(weights=None, include_top=False, input_shape=(224,224,3))
        try:
            base_model.load_weights(base_weights_path)
        except Exception as e:
            print('Warning: could not load base weights into tuner backbone:', e)

        fine_tune_at = hp.Choice('fine_tune_at', [140, 170])
        base_model.trainable = True
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False

        inputs = tf.keras.Input(shape=(224,224,3))
        x = base_model(inputs, training=False)
        x = layers.GlobalAveragePooling2D()(x)

        units_l1 = hp.Choice('units_l1', [512,128,64])
        dropout_rate = hp.Float('dropout_rate', 0.1, 0.4, step=0.1)
        x = layers.Dense(units_l1, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(32, activation='relu')(x)
        outputs = layers.Dense(1, activation='linear')(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        lr = hp.Choice('learning_rate', [1e-7, 1e-6, 5e-6])
        opt_name = hp.Choice('optimizer', ['Adam','RMSprop'])
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr) if opt_name=='RMSprop' else tf.keras.optimizers.Adam(learning_rate=lr)

        model.compile(optimizer=opt, loss='mse', metrics=['mae','mse'])
        return model

    tuner = kt.RandomSearch(build_model_for_tuning, objective='val_mae', max_trials=max_trials, executions_per_trial=executions_per_trial, directory=str(output_dir), project_name='resnet_finetune_tuning')
    tuner.search(train_ds, validation_data=val_ds, epochs=10, verbose=1)
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    return tuner, best_hp


**Penjelasan sebelum sel Final training & evaluasi**

Sel ini memuat fungsi `final_train_with_best_hp()` dan `evaluate_model()` untuk: memuat bobot backbone, mengunci layer sampai `fine_tune_at`, melatih lagi sesuai HP terbaik, dan mengevaluasi model pada test set.

In [7]:
# 7) Final train with best HP and evaluation

def final_train_with_best_hp(base_weights_path, best_hp, train_ds, val_ds, initial_epochs=30, fine_tune_epochs=40, output_dir=Path('.') ):
    units = best_hp.get('units_l1')
    dropout = best_hp.get('dropout_rate')
    lr = best_hp.get('learning_rate')
    opt_name = best_hp.get('optimizer')
    fine_tune_at = best_hp.get('fine_tune_at')

    model, base_model = build_regression_model(units_l1=units, dropout_rate=dropout, base_trainable=True)
    try:
        base_model.load_weights(base_weights_path)
    except Exception as e:
        print('Warning: could not load base weights into final model:', e)

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    opt = tf.keras.optimizers.RMSprop(learning_rate=lr) if opt_name=='RMSprop' else tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='mse', metrics=['mae','mse'])

    callbacks = [
        EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-8, verbose=1),
        ModelCheckpoint(str(output_dir / 'best_final.keras'), monitor='val_loss', save_best_only=True, verbose=1)
    ]

    total_epochs = initial_epochs + fine_tune_epochs
    history = model.fit(train_ds, validation_data=val_ds, epochs=total_epochs, initial_epoch=0, callbacks=callbacks, verbose=1)
    return model, history


def evaluate_model(model, test_ds):
    y_true = []
    y_pred = []
    for x,y in test_ds:
        preds = model.predict(x, verbose=0).flatten()
        y_pred.extend(preds.tolist())
        y_true.extend(y.numpy().flatten().tolist())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {'mae': float(mae), 'mse': float(mse), 'rmse': float(rmse), 'r2': float(r2)}


**Penjelasan sebelum sel Parameter & Run example**

Sel parameter di bawah berisi variabel yang harus Anda sesuaikan sebelum menjalankan pipeline (lokasi data, ukuran gambar, batch size, opsi tuning, seed, dsb.). Setelah disesuaikan, jalankan sel-sel dari atas ke bawah.

## Outline & menjalankan per tahapan

Notebook sudah diatur per bagian (Imports, Utilities, tf.data builder, Model builder, Phase 1, Keras Tuner, Final training, Parameter). Gunakan sel Parameter (akhir) untuk mengontrol eksekusi per-tahapan melalui flag RUN_*. Jika ingin menjalankan manual, jalankan sel yang diperlukan secara langsung.

In [8]:
# 8) Parameters & run example (edit then run in order)

# Stage control flags: set True untuk mengeksekusi tahapan ini saat menjalankan SEL PARAMETER
RUN_COLLECT = True        # collect & stratified split
RUN_OVERSAMPLE = True     # oversample train only (resample filepaths)
RUN_DATASET = True        # build tf.data datasets
RUN_PHASE1 = True         # train phase1 (head, backbone frozen)
RUN_SAVE_BASE = True      # save base_model weights after Phase1
RUN_TUNER = False         # jalankan Keras Tuner
RUN_FINAL = True          # final training & evaluate

# Parameters (edit as needed)
DATA_DIR = Path('data valid merah')
OUTPUT_DIR = Path('outputs_compact')
IMG_SIZE = 224
BATCH_SIZE = 32
INITIAL_EPOCHS = 30
PHASE1_LR = 1e-6
FINE_TUNE_EPOCHS = 40
OVERSAMPLE = True
TARGET_COUNT = 450
SEED = 42
DO_TUNING = RUN_TUNER  # internal convenience
MAX_TRIALS = 20
EXECUTIONS_PER_TRIAL = 1

# reproducibility seeds
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 1) collect and split
if RUN_COLLECT:
    filepaths, labels = collect_image_paths(DATA_DIR)
    print('Total images found:', len(filepaths))
    (fp_train, lb_train), (fp_val, lb_val), (fp_test, lb_test) = stratified_split(
        filepaths, labels, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=SEED
    )
    print('Counts - train:', len(fp_train), 'val:', len(fp_val), 'test:', len(fp_test))
else:
    required = ['fp_train','lb_train','fp_val','lb_val','fp_test','lb_test']
    if not all(name in globals() for name in required):
        raise RuntimeError('Skipping collect but split variables are missing. Set RUN_COLLECT=True or define split variables manually.')

# 2) oversample train only (resample filepaths)
if RUN_OVERSAMPLE:
    fp_train_os, lb_train_os = oversample_filepaths(fp_train, lb_train, target_count=TARGET_COUNT, seed=SEED)
else:
    fp_train_os, lb_train_os = fp_train, lb_train

# 3) datasets
if RUN_DATASET:
    y_train = labels_to_ppb(lb_train_os)
    y_val = labels_to_ppb(lb_val)
    y_test = labels_to_ppb(lb_test)

    train_ds = build_dataset_from_paths(fp_train_os, y_train, img_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE, augment=True, shuffle=True)
    val_ds = build_dataset_from_paths(fp_val, y_val, img_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE, augment=False, shuffle=False)
    test_ds = build_dataset_from_paths(fp_test, y_test, img_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE, augment=False, shuffle=False)
else:
    if 'train_ds' not in globals():
        raise RuntimeError('Skipping dataset build but dataset variables not found. Set RUN_DATASET=True or define train_ds/val_ds/test_ds manually.')

# 4) phase1 train
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
if RUN_PHASE1:
    model, base_model = build_regression_model(input_shape=(IMG_SIZE,IMG_SIZE,3), backbone='resnet50', units_l1=512, dropout_rate=0.3, base_trainable=False)
    history_p1 = train_phase1(model, train_ds, val_ds, initial_epochs=INITIAL_EPOCHS, lr=PHASE1_LR, output_dir=OUTPUT_DIR)
else:
    if 'base_model' not in globals():
        print('Phase1 skipped and `base_model` not found in namespace. If you plan to run tuner/final, ensure `base_weights_path` exists or set RUN_PHASE1=True to produce it.')

# 5) save backbone weights for tuner/final
base_weights_path = OUTPUT_DIR / 'base_weights.h5'
if RUN_SAVE_BASE:
    if 'base_model' in globals():
        save_base_weights(base_model, base_weights_path)
        print('Saved backbone weights to', base_weights_path)
    else:
        print('Cannot save base weights: `base_model` not defined. Run Phase1 first or set base_weights_path manually.')

# 6) tuner (optional)
if RUN_TUNER:
    if not DO_TUNING:
        raise RuntimeError('RUN_TUNER=True but keras-tuner not available or DO_TUNING flag mismatch.')
    tuner, best_hp = run_keras_tuner(train_ds, val_ds, str(base_weights_path), max_trials=MAX_TRIALS, executions_per_trial=EXECUTIONS_PER_TRIAL, output_dir=OUTPUT_DIR)
else:
    class DummyHP(dict):
        def get(self, k, default=None):
            return {'units_l1':512, 'dropout_rate':0.3, 'learning_rate':5e-6, 'optimizer':'Adam', 'fine_tune_at':140}.get(k, default)
    best_hp = DummyHP()

# 7) final train and evaluate
if RUN_FINAL:
    final_model, history_final = final_train_with_best_hp(str(base_weights_path), best_hp, train_ds, val_ds, initial_epochs=INITIAL_EPOCHS, fine_tune_epochs=FINE_TUNE_EPOCHS, output_dir=OUTPUT_DIR)
    metrics = evaluate_model(final_model, test_ds)
    print('Test metrics:', metrics)

    # 8) save final model
    final_model.save(OUTPUT_DIR / 'aflatoxin_final_model.keras')
    print('Saved final model and artifacts in', OUTPUT_DIR)
else:
    print('Final training skipped.')


Total images found: 871
Counts - train: 609 val: 174 test: 88
Epoch 1/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step - loss: 6.8126 - mae: 2.3227 - mse: 6.8126
Epoch 1: val_loss improved from inf to 4.91202, saving model to outputs_compact\best_phase1.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 53s 860ms/step - loss: 6.8093 - mae: 2.3220 - mse: 6.8093 - val_loss: 4.9120 - val_mae: 2.0233 - val_mse: 4.9120 - learning_rate: 1.0000e-06
Epoch 2/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 788ms/step - loss: 6.2010 - mae: 2.1939 - mse: 6.2010
Epoch 2: val_loss improved from 4.91202 to 4.18830, saving model to outputs_compact\best_phase1.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 50s 872ms/step - loss: 6.1967 - mae: 2.1930 - mse: 6.1967 - val_loss: 4.1883 - val_mae: 1.8385 - val_mse: 4.1883 - learning_rate: 1.0000e-06
Epoch 3/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - loss: 5.5528 - mae: 2.0306 - mse: 5.5528
Epoch 3: val_loss improved from 4.18830 to 3.55785, saving model to outputs_compact\best_phase1.keras
57/57 ━━━━━━━━━

ValueError: The filename must end in `.weights.h5`. Received: filepath=outputs_compact\base_weights.h5